In [1]:
import streamlit as st
import pydeck as pdk
import json
import pandas as pd
import random
import geopandas as gpd


## Creating the dummy user interface for the model
Current problems include:
- hexagons not side by side
- map still appearing when running the py file but not in Jupyter

In [7]:
#downloaded from https://martinjc.github.io/UK-GeoJSON/ *ENGLAND > National > \
# Local Authority Districts and converted to GEOJson

with open("England.geojson", "r") as file:
    data = json.load(file)


In [4]:
#Changing the centroid lons and lats as a DataFrame
gdf = gpd.GeoDataFrame.from_features(data["features"])
coordinates = gdf.get_coordinates()
coordinates.columns = ["lng", "lat"]
centroids = gdf.centroid.to_frame()b
df = pd.DataFrame()
df["lng"] = gdf.centroid.x
df["lat"] = gdf.centroid.y


In [8]:
#styling
layer = pdk.Layer(
    "HexagonLayer",
    df,
    get_position=["lng", "lat"],
    elevation_scale=0,
    pickable=True,
    elevation_range=[0, 3000],
    filled=True,
    coverage=10,
)


In [9]:
view_state = pdk.ViewState(
    longitude=-1.415,
    latitude=52.2323,
    zoom=5,
    min_zoom=5,
    max_zoom=15,
    pitch=0,
    bearing=0,
)


In [10]:
# Render without the map
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    map_style="",
    map_provider="",
    api_keys=None,
)


In [13]:
# Streamlit app layout with sliders and titles
st.title("UK, hun?")

# Create a placeholder for the map on the left side
st.sidebar.header("Interactive Hex Map")

# Create sliders for conservative and labor party approval ratings on the right side
st.sidebar.header("Sliders for Polling/Approval Ratings")

conservative_rating = st.sidebar.slider(
    "Conservative Rating", min_value=0, max_value=100, value=50
)
labor_party_rating = st.sidebar.slider(
    "Labor Party Rating", min_value=0, max_value=100, value=50
)

# Display the current values of the sliders
st.sidebar.text(f"Current Conservative Rating: {conservative_rating}%")
st.sidebar.text(f"Current Labor Party Rating: {labor_party_rating}%")

# Display the PyDeck chart using st.pydeck_chart
st.pydeck_chart(r)


DeltaGenerator()

## Testing a new layer style

In [14]:
#new layerstyle using h3_hex
h3_layer = pdk.Layer(
    "H3HexagonLayer",
    df,
    pickable=True,
    stroked=True,
    filled=True,
    extruded=False,
    get_hexagon="hex",
    get_fill_color="[255 - count, 255, count]",
    get_line_color=[255, 255, 255],
    line_width_min_pixels=2,
)

# Set the viewport location
view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=14, bearing=0, pitch=30)


# Render
r = pdk.Deck(layers=[h3_layer], initial_view_state=view_state, tooltip={"text": "Count: {count}"})
r.to_html("h3_hexagon_layer.html")


In [ ]:

# Streamlit app layout
st.title("UK, hun?")

# Create a placeholder for the map on the left side
st.sidebar.header("Interactive Hex Map")

# Create sliders for conservative and labor party approval ratings on the right side
st.sidebar.header("Sliders for Polling/Approval Ratings")
conservative_rating = st.sidebar.slider(
    "Conservative Rating", min_value=0, max_value=100, value=50
)
labor_party_rating = st.sidebar.slider(
    "Labor Party Rating", min_value=0, max_value=100, value=50
)

# Display the current values of the sliders
st.sidebar.text(f"Current Conservative Rating: {conservative_rating}%")
st.sidebar.text(f"Current Labor Party Rating: {labor_party_rating}%")

# Display the PyDeck chart using st.pydeck_chart
st.pydeck_chart(r)
